# Defining Your Interfaces


In [16]:
import decimal
from dataclasses import dataclass
from enum import auto, Enum
from typing import Iterable, Optional, List
from copy import deepcopy
from contextlib import contextmanager


## Natural Interface Design


### Natural Interfaces in Action


Suppose we want to design an interface for part of an automated grocery pick-up service.


In [2]:
# We can represent recipes as follows

class ImperialMeasure(Enum):
    TEASPOON = auto()
    TABLESPOON = auto()
    CUP = auto()

@dataclass(frozen=True)
class Ingredient:
    name: str
    brand: str
    amount: float = 1
    units: ImperialMeasure = ImperialMeasure.CUP

@dataclass
class Recipe:
    name: str
    ingredients: list[Ingredient]
    servings: int


In [3]:
# We have also functions and types to retrieve local grocery store inventory:

class Coordinates:
    pass

@dataclass(frozen=True)
class Store:
    coordinates: Coordinates
    name: str

@dataclass(frozen=True)
class Item:
    name: str
    brand: str
    measure: ImperialMeasure
    price_in_cents: decimal.Decimal
    amount: float

Inventory = dict[Store, List[Item]]

def get_grocery_inventory() -> Inventory:
    # reach out to APIs and populate the dictionary
    # ... snip ...
    return

def reserve_items(store: Store, items: Iterable[Item]) -> bool:
    # ... snip ...
    return

def unreserve_items(store: Store, items: Iterable[Item]) -> bool:
    # ... snip ...
    return

def order_items(store: Store, items: Iterable[Item]) -> bool:
    # ... snip ...
    return


In [4]:
# The first (incomplete) iteration of the calling code, that generates the ingredient list 
# to order from each grocery store, looks like the following

def get_recipes_from_scans():
    return 

def display_order(order):
    return 

def wait_for_user_order_confirmation():
    return 

def wait_for_user_grocery_confirmation():
    return 

def deliver_ingredients():
    return 

recipes: List[Recipe] = get_recipes_from_scans()

# We need to do something here to get the order
order = '????'

# the user can make changes if needed
display_order(order) # TODO once we know what an order is
wait_for_user_order_confirmation()
if order.is_confirmed():
    grocery_inventory = get_grocery_inventory()
    # HELP, what do we do with ingredients now that we have grocery inventory
    grocery_list = '????'
    # HELP we need to do some reservation of ingredients so others
    # don't take them
    wait_for_user_grocery_confirmation(grocery_list)
    # HELP - actually order the ingredients ????
    deliver_ingredients(grocery_list)


AttributeError: 'str' object has no attribute 'is_confirmed'

In [5]:
# To complete the last code, let's implement the Order class

class Order:
    ''' An Order class that represents a list of ingredients '''
    def __init__(self, recipes: Iterable[Recipe]):
        self.__ingredients: set[Ingredient] = set()
        for recipe in recipes:
            for ingredient in recipe.ingredients:
                self.add_ingredient(ingredient)
    
    def get_ingredients(self) -> list[Ingredient]:
        ''' Return a alphabetically sorted list of ingredients '''
        # Return a copy so that users won't inadvertently mess with
        # our internal data
        return sorted(
            deepcopy(self.__ingredients),
            key=lambda ing: ing.name
        )
        
    def _get_matching_ingredient(
        self,
        ingredient: Ingredient
    ) -> Optional[Ingredient]:
        try:
            return next(
                ing 
                for ing in self.__ingredients 
                if ((ing.name, ing.brand) == (ingredient.name, ingredient.brand))
            )
        except StopIteration:
            return None

    def add_ingredient(self, ingredient: Ingredient):
        '''
        Adds the ingredient if it's not already added, or increases the amount if it has
        '''
        target_ingredient = self._get_matching_ingredient(ingredient)
        if target_ingredient is None:
            # ingredient for the first time - add it
            self.__ingredients.add(ingredient)
        else:
            # add ingredient to existing set
            '????'
            pass



In [6]:
# But we need to conserve the following invariant: if the order is confirmed, 
# a user should not be able to modify anything inside it. 
# So the  Order class will be changed to do the following:

class OrderAlreadyFinalizedError(RuntimeError):
    # Inheriting from RuntimeError to allow users to provide a message
    # when raising this exception
    pass


class Order:
    '''
    An Order class that represents a list of ingredients
    Once confirmed, it cannot be modified
    '''
    def __init__(self, recipes: Iterable[Recipe]):
        self.__confirmed = False
        # ... snip ...
        # ... snip ...

    def add_ingredient(self, ingredient: Ingredient):
        self.__disallow_modification_if_confirmed()
        # ... snip ...

    def __disallow_modification_if_confirmed(self):
        if self.__confirmed:
            raise OrderAlreadyFinalizedError(
                'Order is confirmed -changing it is not allowed'
            )

    def confirm(self):
        self.__confirmed = True

    def unconfirm(self):
        self.__confirmed = False

    def is_confirmed(self):
        return self.__confirmed


### Magic Methods


In [7]:
# If we want to add instances of the Ingreditent class, we can use the magic method __add__ 
# to control behavior for addition, as follows

@dataclass(frozen=True)
class Ingredient:
    name: str
    brand: str
    amount: float = 1
    units: ImperialMeasure = ImperialMeasure.CUP

    def __add__(self, rhs: Ingredient):
        # make sure we are adding the same ingredient
        assert (self.name, self.brand) == (rhs.name, rhs.brand)

        # build up conversion chart (lhs, rhs): multiplication factor
        conversion: dict[tuple[ImperialMeasure, ImperialMeasure], float] = {
            (ImperialMeasure.CUP, ImperialMeasure.CUP): 1,
            (ImperialMeasure.CUP, ImperialMeasure.TABLESPOON): 16,
            (ImperialMeasure.CUP, ImperialMeasure.TEASPOON): 48,
            (ImperialMeasure.TABLESPOON, ImperialMeasure.CUP): 1/16,
            (ImperialMeasure.TABLESPOON, ImperialMeasure.TABLESPOON): 1,
            (ImperialMeasure.TABLESPOON, ImperialMeasure.TEASPOON): 3,
            (ImperialMeasure.TEASPOON, ImperialMeasure.CUP): 1/48,
            (ImperialMeasure.TEASPOON, ImperialMeasure.TABLESPOON): 1/3,
            (ImperialMeasure.TEASPOON, ImperialMeasure.TEASPOON): 1
        }

        return Ingredient(
            rhs.name,
            rhs.brand,
            rhs.amount + self.amount * conversion[(rhs.units, self.units)],
            rhs.units
        )


In [21]:
# Now with the __add__ method defined for Ingredient, we can implement the
# add_ingredient() method as follows 

class OrderWith1Method:
    #...
    
    def add_ingredient(self, ingredient: Ingredient):
        '''
        Adds the ingredient if it's not already added, or increases the amount if it has
        '''
        target_ingredient = self._get_matching_ingredient(ingredient)
        if target_ingredient is None:
            # ingredient for the first time - add it
            self.__ingredients.add(ingredient)
        else:
            # add ingredient to existing set
            target_ingredient += ingredient    

    #...


### Context Managers


In [12]:
# The second iteration of the calling code looks as follows:

class GroceryList:
    def __init__(self, order, grocery_inventory) -> None:
        self.order = order 
        self.grocery_inventory = grocery_inventory

    def reserve_items_from_stores():
        return
    
    def is_confirmed():
        return 
    
    def order_and_unreserve_items():
        return

    def unreserve_items():
        return

order = Order(recipes)
display_order(order) # The user can make changes if needed
wait_for_user_order_confirmation()

if order.is_confirmed():
    grocery_inventory = get_grocery_inventory()
    grocery_list = GroceryList(order, grocery_inventory)
    grocery_list.reserve_items_from_stores()
    wait_for_user_grocery_confirmation(grocery_list)
    
    if grocery_list.is_confirmed():
        grocery_list.order_and_unreserve_items()
        deliver_ingredients(grocery_list)
    else:
        grocery_list.unreserve_items()


In [15]:
# But what if there's an exception in one of the last function calls? 
# In that case, it'd be nice to make the grocery list unreserve items automatically 
# in case of error, which can be done with a context manager

@contextmanager
def create_grocery_list(order: Order, inventory: Inventory):
    # _GroceryList can be also used if you want users to instantiate the class 
    # only with this function
    grocery_list = GroceryList(order, inventory) 
    try:
        yield grocery_list
    finally:
        if grocery_list.has_reserved_items():
            grocery_list.unreserve_items()


In [19]:
# The last context manager can be used in the calling code as follows 

if order.is_confirmed():
    grocery_inventory = get_grocery_inventory()
    
    with create_grocery_list(order, grocery_inventory) as grocery_list:
        grocery_list.reserve_items_from_stores()
        wait_for_user_grocery_confirmation(grocery_list)
        grocery_list.order_and_unreserve_items()
    
    deliver_ingredients(grocery_list)
